<a href="https://colab.research.google.com/github/gaurav1610/E-commerce-Analytics/blob/master/E_commerce_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Customer Gender

## Importing Package

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
#Use LogisticRegression for classifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#oversampling
from imblearn.over_sampling import SMOTE
from collections import Counter


### Importing dataset

In [426]:
cd /content/drive/My Drive/ML/E-commerce_data

/content/drive/My Drive/ML/E-commerce_data


In [0]:
train=pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
full_data=pd.concat([train,test],axis=0,ignore_index=True)


In [428]:
full_data.tail()

,session_id,startTime,endTime,ProductList,gender
14995,u23966,15/12/14 21:11,15/12/14 21:12,A00001/B00031/C00044/D03435/;A00001/B00031/C00...,NaN
14996,u20527,02/12/14 7:53,02/12/14 7:54,A00002/B00002/C00004/D17474/;A00002/B00002/C00...,NaN
14997,u13253,15/11/14 21:38,15/11/14 21:46,A00002/B00007/C00098/D03350/;A00002/B00007/C00...,NaN
14998,u17094,17/12/14 8:51,17/12/14 8:51,A00001/B00031/C00044/D09604/,NaN
14999,u24310,17/12/14 17:25,17/12/14 17:26,A00002/B00002/C00003/D20004/;A00002/B00002/C00...,NaN


### Converting startTime and endTime into datetime format and splitting them into columns


In [429]:
full_data['startTime'] = pd.to_datetime(full_data['startTime'])
full_data['endTime'] = pd.to_datetime(full_data['endTime'])
full_data['duration'] = full_data['endTime'] - full_data['startTime']
full_data['duration'] = full_data['duration'].astype('timedelta64[m]')
full_data['weekday'] = full_data['startTime'].dt.dayofweek
full_data['hour_24h'] = full_data['startTime'].dt.hour
full_data.tail()

,session_id,startTime,endTime,ProductList,gender,duration,weekday,hour_24h
14995,u23966,2014-12-15 21:11:00,2014-12-15 21:12:00,A00001/B00031/C00044/D03435/;A00001/B00031/C00...,NaN,1.0,0,21
14996,u20527,2014-02-12 07:53:00,2014-02-12 07:54:00,A00002/B00002/C00004/D17474/;A00002/B00002/C00...,NaN,1.0,2,7
14997,u13253,2014-11-15 21:38:00,2014-11-15 21:46:00,A00002/B00007/C00098/D03350/;A00002/B00007/C00...,NaN,8.0,5,21
14998,u17094,2014-12-17 08:51:00,2014-12-17 08:51:00,A00001/B00031/C00044/D09604/,NaN,0.0,2,8
14999,u24310,2014-12-17 17:25:00,2014-12-17 17:26:00,A00002/B00002/C00003/D20004/;A00002/B00002/C00...,NaN,1.0,2,17


In [0]:
full_data['duration']=full_data.duration.astype(int)

In [431]:
full_data.shape

(15000, 8)

In [0]:
# full_data.set_index('session_id',inplace=False)

### Extracting category,sub_category,sub_sub_category,Product_code form Product List

In [433]:
full_data.ProductList[1]

'A00001/B00009/C00031/D29404/;A00001/B00009/C00031/D02617/;A00001/B00009/C00031/D29407/;A00001/B00009/C00031/D29410/;A00001/B00009/C00031/D29411/;A00001/B00009/C00031/D25444/;A00001/B00009/C00031/D29418/'

In [434]:
full_data.tail()

,session_id,startTime,endTime,ProductList,gender,duration,weekday,hour_24h
14995,u23966,2014-12-15 21:11:00,2014-12-15 21:12:00,A00001/B00031/C00044/D03435/;A00001/B00031/C00...,NaN,1,0,21
14996,u20527,2014-02-12 07:53:00,2014-02-12 07:54:00,A00002/B00002/C00004/D17474/;A00002/B00002/C00...,NaN,1,2,7
14997,u13253,2014-11-15 21:38:00,2014-11-15 21:46:00,A00002/B00007/C00098/D03350/;A00002/B00007/C00...,NaN,8,5,21
14998,u17094,2014-12-17 08:51:00,2014-12-17 08:51:00,A00001/B00031/C00044/D09604/,NaN,0,2,8
14999,u24310,2014-12-17 17:25:00,2014-12-17 17:26:00,A00002/B00002/C00003/D20004/;A00002/B00002/C00...,NaN,1,2,17


In [0]:
temp = full_data['ProductList'].str.split(';')

In [0]:
full_data = full_data.reindex(full_data.index.repeat(temp.apply(len)))

In [0]:
full_data['product_data'] = np.hstack(temp)

In [438]:
full_data.head()

,session_id,startTime,endTime,ProductList,gender,duration,weekday,hour_24h,product_data
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,1,0,18,A00002/B00003/C00006/D28435/
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,1,0,18,A00002/B00003/C00006/D02554/
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,1,0,18,A00002/B00003/C00006/D28436/
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,1,0,18,A00002/B00003/C00006/D28437/
1,u10253,2014-12-16 14:35:00,2014-12-16 14:41:00,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male,6,1,14,A00001/B00009/C00031/D29404/


In [0]:
full_data['category'] = full_data['product_data'].str.split('/').str[0]
full_data['sub_category'] = full_data['product_data'].str.split('/').str[1]
full_data['sub_sub_category'] = full_data['product_data'].str.split('/').str[2]
full_data['product'] = full_data['product_data'].str.split('/').str[3]

In [440]:
full_data.head()

,session_id,startTime,endTime,ProductList,gender,duration,weekday,hour_24h,product_data,category,sub_category,sub_sub_category,product
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,1,0,18,A00002/B00003/C00006/D28435/,A00002,B00003,C00006,D28435
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,1,0,18,A00002/B00003/C00006/D02554/,A00002,B00003,C00006,D02554
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,1,0,18,A00002/B00003/C00006/D28436/,A00002,B00003,C00006,D28436
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,1,0,18,A00002/B00003/C00006/D28437/,A00002,B00003,C00006,D28437
1,u10253,2014-12-16 14:35:00,2014-12-16 14:41:00,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male,6,1,14,A00001/B00009/C00031/D29404/,A00001,B00009,C00031,D29404


In [0]:
drop_lst = ['startTime', 'endTime', 'ProductList','product_data']

In [0]:
full_data = full_data.drop(drop_lst,axis =1 )

In [443]:
full_data.head()

,session_id,gender,duration,weekday,hour_24h,category,sub_category,sub_sub_category,product
0,u16159,female,1,0,18,A00002,B00003,C00006,D28435
0,u16159,female,1,0,18,A00002,B00003,C00006,D02554
0,u16159,female,1,0,18,A00002,B00003,C00006,D28436
0,u16159,female,1,0,18,A00002,B00003,C00006,D28437
1,u10253,male,6,1,14,A00001,B00009,C00031,D29404


In [0]:
df=full_data[['category','sub_category','sub_sub_category','product']]

In [0]:
df=pd.get_dummies(df)

In [446]:
df.head()

,category_A00001,category_A00002,category_A00003,category_A00004,category_A00005,category_A00006,category_A00007,category_A00008,category_A00009,category_A00010,category_A00011,sub_category_B00001,sub_category_B00002,sub_category_B00003,sub_category_B00004,sub_category_B00005,sub_category_B00006,sub_category_B00007,sub_category_B00008,sub_category_B00009,sub_category_B00010,sub_category_B00011,sub_category_B00012,sub_category_B00013,sub_category_B00014,sub_category_B00015,sub_category_B00016,sub_category_B00017,sub_category_B00018,sub_category_B00019,sub_category_B00020,sub_category_B00021,sub_category_B00022,sub_category_B00023,sub_category_B00024,sub_category_B00025,sub_category_B00026,sub_category_B00027,sub_category_B00028,sub_category_B00029,...,product_D36074,product_D36075,product_D36076,product_D36077,product_D36078,product_D36079,product_D36080,product_D36081,product_D36082,product_D36083,product_D36084,product_D36085,product_D36087,product_D36088,product_D36089,product_D36090,product_D36091,product_D36093,product_D36095,product_D36096,product_D36098,product_D36099,product_D36100,product_D36101,product_D36102,product_D36104,product_D36105,product_D36106,product_D36107,product_D36108,product_D36109,product_D36110,product_D36111,product_D36112,product_D36113,product_D36114,product_D36115,product_D36116,product_D36117,product_D36122
0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
final_df=pd.concat([full_data[['session_id','duration','weekday','hour_24h','gender']],df],axis=1)

In [448]:
final_df.head()

,session_id,duration,weekday,hour_24h,gender,category_A00001,category_A00002,category_A00003,category_A00004,category_A00005,category_A00006,category_A00007,category_A00008,category_A00009,category_A00010,category_A00011,sub_category_B00001,sub_category_B00002,sub_category_B00003,sub_category_B00004,sub_category_B00005,sub_category_B00006,sub_category_B00007,sub_category_B00008,sub_category_B00009,sub_category_B00010,sub_category_B00011,sub_category_B00012,sub_category_B00013,sub_category_B00014,sub_category_B00015,sub_category_B00016,sub_category_B00017,sub_category_B00018,sub_category_B00019,sub_category_B00020,sub_category_B00021,sub_category_B00022,sub_category_B00023,sub_category_B00024,...,product_D36074,product_D36075,product_D36076,product_D36077,product_D36078,product_D36079,product_D36080,product_D36081,product_D36082,product_D36083,product_D36084,product_D36085,product_D36087,product_D36088,product_D36089,product_D36090,product_D36091,product_D36093,product_D36095,product_D36096,product_D36098,product_D36099,product_D36100,product_D36101,product_D36102,product_D36104,product_D36105,product_D36106,product_D36107,product_D36108,product_D36109,product_D36110,product_D36111,product_D36112,product_D36113,product_D36114,product_D36115,product_D36116,product_D36117,product_D36122
0,u16159,1,0,18,female,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,u16159,1,0,18,female,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,u16159,1,0,18,female,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,u16159,1,0,18,female,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,u10253,6,1,14,male,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
train_f=final_df[final_df.gender.notnull()]

In [450]:
train_f.head()

,session_id,duration,weekday,hour_24h,gender,category_A00001,category_A00002,category_A00003,category_A00004,category_A00005,category_A00006,category_A00007,category_A00008,category_A00009,category_A00010,category_A00011,sub_category_B00001,sub_category_B00002,sub_category_B00003,sub_category_B00004,sub_category_B00005,sub_category_B00006,sub_category_B00007,sub_category_B00008,sub_category_B00009,sub_category_B00010,sub_category_B00011,sub_category_B00012,sub_category_B00013,sub_category_B00014,sub_category_B00015,sub_category_B00016,sub_category_B00017,sub_category_B00018,sub_category_B00019,sub_category_B00020,sub_category_B00021,sub_category_B00022,sub_category_B00023,sub_category_B00024,...,product_D36074,product_D36075,product_D36076,product_D36077,product_D36078,product_D36079,product_D36080,product_D36081,product_D36082,product_D36083,product_D36084,product_D36085,product_D36087,product_D36088,product_D36089,product_D36090,product_D36091,product_D36093,product_D36095,product_D36096,product_D36098,product_D36099,product_D36100,product_D36101,product_D36102,product_D36104,product_D36105,product_D36106,product_D36107,product_D36108,product_D36109,product_D36110,product_D36111,product_D36112,product_D36113,product_D36114,product_D36115,product_D36116,product_D36117,product_D36122
0,u16159,1,0,18,female,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,u16159,1,0,18,female,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,u16159,1,0,18,female,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,u16159,1,0,18,female,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,u10253,6,1,14,male,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
test_f=final_df[final_df.gender.isnull()]

In [452]:
test_f.head()

,session_id,duration,weekday,hour_24h,gender,category_A00001,category_A00002,category_A00003,category_A00004,category_A00005,category_A00006,category_A00007,category_A00008,category_A00009,category_A00010,category_A00011,sub_category_B00001,sub_category_B00002,sub_category_B00003,sub_category_B00004,sub_category_B00005,sub_category_B00006,sub_category_B00007,sub_category_B00008,sub_category_B00009,sub_category_B00010,sub_category_B00011,sub_category_B00012,sub_category_B00013,sub_category_B00014,sub_category_B00015,sub_category_B00016,sub_category_B00017,sub_category_B00018,sub_category_B00019,sub_category_B00020,sub_category_B00021,sub_category_B00022,sub_category_B00023,sub_category_B00024,...,product_D36074,product_D36075,product_D36076,product_D36077,product_D36078,product_D36079,product_D36080,product_D36081,product_D36082,product_D36083,product_D36084,product_D36085,product_D36087,product_D36088,product_D36089,product_D36090,product_D36091,product_D36093,product_D36095,product_D36096,product_D36098,product_D36099,product_D36100,product_D36101,product_D36102,product_D36104,product_D36105,product_D36106,product_D36107,product_D36108,product_D36109,product_D36110,product_D36111,product_D36112,product_D36113,product_D36114,product_D36115,product_D36116,product_D36117,product_D36122
10500,u12112,0,1,13,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10501,u19725,0,4,13,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10502,u11795,0,6,10,NaN,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10503,u22639,3,1,20,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10503,u22639,3,1,20,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X=train_f.drop(['gender','session_id'],axis=1)

In [0]:
Y =  train_f['gender'].map({'female':1,'male':0})

### **Train-Validation Splitting**:Further splitting train data into training and validation data 

In [0]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3)

In [456]:
print(X.shape)
print(X_train.shape)
print(X_val.shape)

(23251, 22363)
(16275, 22363)
(6976, 22363)


## Tuning & selecting Model 

In [0]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label = Y_train)

#setting parameters for lightgbm
param = {'num_leaves':30, 'objective':'binary','max_depth':8,'learning_rate':.11, 'max_bin' : 200}
param['metric'] = ['auc', 'accuracy']

clf2 = lgb.train(param, train_data, 4000, verbose_eval=True)

In [458]:
clf = LogisticRegression(class_weight = {1:.25, 0:.35},max_iter=500)
clf.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight={0: 0.35, 1: 0.25}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [459]:
print("Evalute based on validation set")
print("f1 : " + " %s" % f1_score(Y_val, clf.predict(X_val)))
print("accuracy score" + " %s" % accuracy_score(Y_val, clf.predict(X_val)))
print("recall score micro: " + " %s" % recall_score(Y_val, clf.predict(X_val), average='micro'))
print("recall score macro: " + " %s" % recall_score(Y_val, clf.predict(X_val), average='macro'))

Evalute based on validation set
f1 :  0.88384
accuracy score 0.7918577981651376
recall score micro:  0.7918577981651376
recall score macro:  0.5


In [0]:
X_val = X_val.astype(int)
Y_val = Y_val.astype(int)

In [461]:
clf2.predict(X_val).round()

array([1., 1., 1., ..., 1., 0., 1.])

In [462]:
print("Evalute based on validation set")
print("f1 : " + " %s" % f1_score(Y_val, clf2.predict(X_val).round()))
print("accuracy score" + " %s" % accuracy_score(Y_val, clf2.predict(X_val).round()))
# print("recall score micro: " + " %s" % recall_score(Y_val, clf2.predict(X_val)))
# print("recall score macro: " + " %s" % recall_score(Y_val, clf2.predict(X_val), average='macro'))

Evalute based on validation set
f1 :  0.9537826438122586
accuracy score 0.9261754587155964


In [463]:
test_f.head()

,session_id,duration,weekday,hour_24h,gender,category_A00001,category_A00002,category_A00003,category_A00004,category_A00005,category_A00006,category_A00007,category_A00008,category_A00009,category_A00010,category_A00011,sub_category_B00001,sub_category_B00002,sub_category_B00003,sub_category_B00004,sub_category_B00005,sub_category_B00006,sub_category_B00007,sub_category_B00008,sub_category_B00009,sub_category_B00010,sub_category_B00011,sub_category_B00012,sub_category_B00013,sub_category_B00014,sub_category_B00015,sub_category_B00016,sub_category_B00017,sub_category_B00018,sub_category_B00019,sub_category_B00020,sub_category_B00021,sub_category_B00022,sub_category_B00023,sub_category_B00024,...,product_D36074,product_D36075,product_D36076,product_D36077,product_D36078,product_D36079,product_D36080,product_D36081,product_D36082,product_D36083,product_D36084,product_D36085,product_D36087,product_D36088,product_D36089,product_D36090,product_D36091,product_D36093,product_D36095,product_D36096,product_D36098,product_D36099,product_D36100,product_D36101,product_D36102,product_D36104,product_D36105,product_D36106,product_D36107,product_D36108,product_D36109,product_D36110,product_D36111,product_D36112,product_D36113,product_D36114,product_D36115,product_D36116,product_D36117,product_D36122
10500,u12112,0,1,13,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10501,u19725,0,4,13,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10502,u11795,0,6,10,NaN,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10503,u22639,3,1,20,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10503,u22639,3,1,20,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X_test=test_f.drop(['session_id','gender'],axis=1)

In [291]:
X_test.head()

,duration,weekday,hour_24h,category_A00001,category_A00002,category_A00003,category_A00004,category_A00005,category_A00006,category_A00007,category_A00008,category_A00009,category_A00010,category_A00011,sub_category_B00001,sub_category_B00002,sub_category_B00003,sub_category_B00004,sub_category_B00005,sub_category_B00006,sub_category_B00007,sub_category_B00008,sub_category_B00009,sub_category_B00010,sub_category_B00011,sub_category_B00012,sub_category_B00013,sub_category_B00014,sub_category_B00015,sub_category_B00016,sub_category_B00017,sub_category_B00018,sub_category_B00019,sub_category_B00020,sub_category_B00021,sub_category_B00022,sub_category_B00023,sub_category_B00024,sub_category_B00025,sub_category_B00026,...,product_D36074,product_D36075,product_D36076,product_D36077,product_D36078,product_D36079,product_D36080,product_D36081,product_D36082,product_D36083,product_D36084,product_D36085,product_D36087,product_D36088,product_D36089,product_D36090,product_D36091,product_D36093,product_D36095,product_D36096,product_D36098,product_D36099,product_D36100,product_D36101,product_D36102,product_D36104,product_D36105,product_D36106,product_D36107,product_D36108,product_D36109,product_D36110,product_D36111,product_D36112,product_D36113,product_D36114,product_D36115,product_D36116,product_D36117,product_D36122
10500,0,1,13,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10501,0,4,13,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10502,0,6,10,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10503,3,1,20,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10503,3,1,20,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
PREDICTION=clf.predict(X_test)

In [293]:
PREDICTION

array([1, 1, 1, ..., 0, 1, 1])

In [0]:
PREDICTION=PREDICTION.tolist()

In [0]:
prediction=pd.DataFrame()

In [0]:
prediction['gender']=PREDICTION

In [0]:
gender_encoding={1:'female',0:'male'}

In [298]:
gender_encoding

{0: 'male', 1: 'female'}

In [0]:
Y =  []
for i in prediction['gender']:
    Y.append(gender_encoding[i])
    # print(i)

In [0]:
df=pd.DataFrame()
df['session_id']=test_f['session_id']
df['gender']=Y

In [392]:
df.session_id.unique()

array(['u12112', 'u19725', 'u11795', ..., 'u13253', 'u17094', 'u24310'],
      dtype=object)

In [0]:
pred=[]
for i in df.session_id.unique().tolist():
        pred.append(df.loc[df['session_id'] == i, 'gender'].iloc[0])

In [0]:
df1=pd.DataFrame()

In [0]:
df1['session_id']= df.session_id.unique().tolist()
df1['gender'] = pred

In [403]:
df1.head()

,session_id,gender
0,u12112,female
1,u19725,female
2,u11795,female
3,u22639,female
4,u18034,female


In [0]:
df1.to_csv('/content/drive/My Drive/ML/E-commerce_data/sample_submission.csv',index=False)